In [614]:
import config_parser as c
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import KMeans
import nltk.stem 

In [615]:
events = set(['goal','penalty','red','card','yellow','freekick','miss','substitute','scores','save'])

In [616]:
filename='../config/config-swabou.txt'
config = {}

with open(filename,'r') as configfile:
    for line in configfile:
        if line.startswith('#'):
            continue
        name,var=line.partition('=')[::2]
        config[name.strip()]=var.strip()

In [617]:
def get_team_and_player(top_terms):
    home_team_name = config['home-team'].split(':')[0]
    home_team = config['home-team'].split(':')[1].split(',')
    players_h = set(home_team) & set(top_terms) #if player involved is from home team
    d = dict()
    p_val = 100
    player_h = ''
    if len(players_h) > 1:
        for p in players_h:
            p_val_new = top_terms.index(p)
            if p_val_new < p_val:
                player_h = p
                p_val = p_val_new
    elif len(players_h) == 1:
        player_h = next(iter(players_h))
    
    d[home_team_name] = player_h
    away_team_name = config['away-team'].split(':')[0]
    away_team = config['away-team'].split(':')[1].split(',')
    players_a = set(away_team) & set(top_terms) #if player involved is from home team
    p_val = 100
    player_a = ''
    if len(players_a) > 1:
        for p in players_a:
            p_val_new = top_terms.index(p)
            if p_val_new < p_val:
                player_a = p
                p_val = p_val_new
    elif len(players_a) == 1:
        player_a = next(iter(players_a))
    d[away_team_name] = player_a
    return d

In [618]:
def get_event(tweets, n_clusters=1):
    stop_words = set(config['keywords'].lower().split(','))
    stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)\
    .union(set(['http','https','vs']))\
    .union(set(config['home-team'].split(':')[0].split(' ')))\
    .union(set(config['away-team'].split(':')[0].split(' ')))
    vectorizer = TfidfVectorizer(max_df=0.95
                                 ,min_df=1
                                 ,max_features=10000
                                 ,decode_error='replace'
                                 ,stop_words=set(stop_words))
    tfidf=vectorizer.fit_transform(tweets.split(' '))
    model = KMeans(n_clusters=n_clusters).fit(tfidf)
    
    terms = vectorizer.get_feature_names()
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    top_terms = []
    for i in range(n_clusters):
        #print("Cluster %d:" % i)
        for ind in order_centroids[i, :10]:
            top_terms.append(terms[ind])
            print(' %s' % terms[ind]),
            #print ' '
    return set(top_terms) & events, get_team_and_player(top_terms)

In [619]:
def read_spike(spike, filter_lang=False):
    tweet_text = ' '
    with open(spike, 'r') as infile:
        tweets = infile.readlines()
        #print tweets[0],
        minute = tweets[0].split(',')[0].split(':')[1]
        n_tweets = tweets[0].split(',')[3].split(':')[1]
        slope = tweets[0].split(',')[2].split(':')[1]
        for tweet in tweets:
            splits = tweet.split('^')
            if filter_lang:
                if len(splits) == 2 and splits[1].startswith('en'):
                    tweet_text += splits[0]
            else:
                tweet_text += splits[0]
    return tweet_text, minute, int(n_tweets), int(slope)

In [620]:
import glob
spikes = sorted(glob.glob("../results/%s/*.spike"%config['db']))
results = []
for spike in spikes:
    tweet_text, minute, n_tweets,slope = read_spike(spike, True)
    cpt = len(tweet_text)/n_tweets
    if slope > float(config['tpm']) * 0.15:
        print n_tweets, len(tweet_text), cpt,slope,
        print ', %s, %s: '% (spike, minute),
        print get_event(tweet_text)
        print

503 18981 37 195 , ../results/swabou/fh1.spike, 26-28:   gosling  monk  goal  fuck  26  dan  fucking  swans  winning  losing (set(['goal']), {'Swansea': '', 'Bournemouth': u'gosling'})

316 11512 36 135 , ../results/swabou/fh2.spike, 38-40:   shelvey  penalty  level  game  goal  pen  swans  comeback  39  jonjo (set(['penalty', 'goal']), {'Swansea': u'shelvey', 'Bournemouth': ''})

186 7689 41 69 , ../results/swabou/fh3.spike, 10-12:   king  josh  goal  10  lead  southampton  scores  scored  winning  fuck (set(['goal', 'scores']), {'Swansea': '', 'Bournemouth': u'king'})

56 2475 44 55 , ../results/swabou/sh1.spike, 94-95:   city  swans  time  brom  arsenal  ayew  premier  shelvey  everton  king (set([]), {'Swansea': u'ayew', 'Bournemouth': u'king'})



In [621]:
s = set([1,2,4])